### Set path to original pyNeuroChem. Please change to your own path

In [1]:
import sys
sys.path.append('/home/olexandr/notebooks/ASE_ANI/lib')
from ase_interface import ANIENS
from ase_interface import ensemblemolecule

In [2]:
import numpy as np
import  ase
import time
#from ase.build import molecule
#from ase.neb import NEB
#from ase.calculators.mopac import MOPAC
from ase.md.langevin import Langevin
from ase.io.trajectory import Trajectory
from ase import units

from ase.optimize.fire import FIRE as QuasiNewton

from ase.md.nvtberendsen import NVTBerendsen
from ase.md import MDLogger

#from ase.neb import NEBtools
from ase.io import read, write
from ase.optimize import BFGS, LBFGS 

In [3]:
# Load the 
bz = read('data/bz.xyz')

In [4]:
# Set locations of all required network files
wkdir = '../ANI-c08f-ntwk-cv/' # Note the relative path
cnstfile = wkdir + 'rHCNO-4.6A_16-3.1A_a4-8.params' # AEV parameters
saefile  = wkdir + 'sae_6-31gd.dat' # Atomic shifts
nnfdir   = wkdir + '/train' # network prefix
Nn = 5 # Number of networks in the ensemble
gpu = 0 # GPU ID -- this defaults to 0

# Initilize network ensemble
aens = ensemblemolecule(cnstfile, saefile, nnfdir, Nn, gpu)

# Set ANI ensemble calculator
bz.set_calculator(ANIENS(aens))

In [5]:
start_time = time.time()
dyn = LBFGS(bz)
dyn.run(fmax=0.001)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

LBFGS:   0  16:22:03    -6317.899683       0.8240
LBFGS:   1  16:22:03    -6317.911565       0.4601
LBFGS:   2  16:22:03    -6317.920198       0.0773
LBFGS:   3  16:22:03    -6317.920677       0.0446
LBFGS:   4  16:22:03    -6317.921166       0.0126
LBFGS:   5  16:22:03    -6317.921172       0.0019
[ANI Total time: 0.32929229736328125 seconds]


In [6]:
# Temperature
T = 300.0

In [7]:
# We want to run MD with constant energy using the Langevin algorithm
# with a time step of 5 fs, the temperature T and the friction
# coefficient to 0.02 atomic units.
#dyn = Langevin(bz, 2 * units.fs, T * units.kB, 0.02)
dyn = NVTBerendsen(bz, 0.5 * units.fs, 300.0, taut=1.0*1000*units.fs)

In [8]:
def printenergy(a=bz):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    print('Energy per atom: Epot = %.3feV  Ekin = %.3feV (T=%3.0fK)  '
          'Etot = %.3feV' % (epot, ekin, ekin / (1.5 * units.kB), epot + ekin))

In [9]:
dyn.attach(printenergy, interval=100)
#dyn.attach(MDLogger(dyn, bz, 'bz_md_NVT_10ps_1fs.log', header=True, stress=False,
#           peratom=False, mode="w"), interval=50)

In [10]:
printenergy()

Energy per atom: Epot = -526.493eV  Ekin = 0.000eV (T=  0K)  Etot = -526.493eV


In [11]:
start_time = time.time()
dyn.run(2000) # Do 1ps of MD
print('[ANI Total time:', time.time() - start_time, 'seconds]')

/home/jujuman/Gits/ase/ase/md/nvtberendsen.py:66: RuntimeWarning: divide by zero encountered in double_scalars
  (self.temperature / old_temperature - 1.0) *


KeyboardInterrupt: 